## Introduction to Machine Learning  

## Assignment 4: Similarity-based Approaches to Supervised Learning

You can't learn technical subjects without hands-on practice. The assignments are an important part of the course. To submit this assignment you will need to make sure that you save your Jupyter notebook. 

Below are the links of 2 videos that explain:

1. [How to save your Jupyter notebook](https://youtu.be/0aoLgBoAUSA) and,       
2. [How to answer a question in a Jupyter notebook assignment](https://youtu.be/7j0WKhI3W4s).

### Assignment Learning Goals:

By the end of the module, students are expected to:

- Explain the notion of similarity-based algorithms.
- Broadly describe how 𝑘-NNs use distances.
- Describe the effect of using a small/large value of the hyperparameter 𝑘 when using the 𝑘-NN algorithm.
- Explain the problem of curse of dimensionality.
- Explain the general idea of SVMs with RBF kernel.
- Compare and contrast 𝑘-NNs and SVM RBFs.
- Broadly describe the relation of `gamma` and `C` hyperparameters with the fundamental tradeoff.

This assignment covers [Module 4](https://ml-learn.mds.ubc.ca/en/module4) of the online course. You should complete this module before attempting this assignment.

Any place you see `...`, you must fill in the function, variable, or data to complete the code. Substitute the `None` with your completed code and answers then proceed to run the cell!

Note that some of the questions in this assignment will have hidden tests. This means that no feedback will be given as to the correctness of your solution. It will be left up to you to decide if your answer is sufficiently correct. These questions are worth 2 points.

In [1]:
# Import libraries needed for this lab
from hashlib import sha1

import altair as alt
# import graphviz
import numpy as np
import pandas as pd
#from altair_saver import save

from IPython.display import HTML
from sklearn import tree
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neighbors import KNeighborsClassifier
import test_assignment4 as t
#alt.renderers.enable('png')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## 1. Splitting and Exploring Your Data

For the next few questions, we are going to concentrate on wine data obtained from [Kaggle](https://www.kaggle.com/numberswithkartik/red-white-wine-dataset) that examines the different measurements of wine and we will be attempting to predict if each example is of the red or white variety.  

The features in this dataset include: 

- `fixed_acidity`     
- `volatile_acidity`    
- `citric_acid`    
- `residual_sugar`    
- `chlorides`   
- `free_sulfur_dioxide`   
- `total_sulfur_dioxide`    
- `density`    
- `pH`     
- `sulphates`    
- `alcohol`      
- `quality`: (score between 0 and 10)       
- `style`       
     

In [2]:
wine_df = pd.read_csv('data/wine.csv')
wine_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,style
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [3]:
wine_df.shape

(6497, 13)

**Question 1.1** <br> {points: 1}  

How many null values do we have in our dataset? Save your answer in an object named "null_vals" and if necessary, use `dropna()` and save over the object named `wine_df`. 

In [4]:
wine_df.isnull().sum()
null_vals = 0

In [5]:
t.test_1_1(null_vals)

'Success'

**Question 1.2** <br> {points: 1}  

Split the data into 80% train and 20% test sets. Name your training split `train_df` and your test split `test_df`. We want to make sure that everyone has the same split so please specify an input argument of `random_state=2020`.

In [6]:
train_df, test_df = train_test_split(wine_df, test_size = 0.2, random_state = 2020)

In [7]:
t.test_1_2(train_df,test_df)

'Success'

**Question 1.3** <br> {points: 2}  

How many dimensions does this dataset have? Save your answer in an object named `wine_dim`.

In [8]:
wine_df.count(axis='columns')
wine_dim = 13

In [9]:
# check that the variable exists
assert 'wine_dim' in globals(
), "Please make sure that your solution is named 'wine_dim'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.


**Question 1.4** <br> {points: 1}  

Using the `wine_train` data, look at the summary statistics produced by `.describe()` and save the results in an object named `wine_described`.

In [10]:
wine_described = train_df.describe()

In [11]:
t.test_1_4(wine_described)

'Success'

**Question 1.5** <br> {points: 1}  

What is the average pH of the wine for each variant in `train_df`? Save the average red variant pH in an object named `avg_red_ph` and the average white variant as `avg_white_ph`

In [12]:
style_ph = wine_df.groupby('style', as_index=False)['pH'].mean()
avg_red_ph = style_ph.iloc[0,1]
avg_white_ph = style_ph.iloc[1,1]

In [13]:
t.test_1_5(avg_red_ph,avg_white_ph)

'Success'

**Question 1.6** <br> {points: 1}  

What is the average alcohol of the wine styles in `train_df`? Save the average alcohol content of the red variant in an object named `avg_red_alc` and the average alcohol content in the white variant as `avg_white_alc`.

In [14]:
style_alc = wine_df.groupby('style', as_index=False)['alcohol'].mean()
avg_red_alc = style_alc.iloc[0,1]
avg_white_alc = style_alc.iloc[1,1]

In [15]:
t.test_1_6(avg_red_alc,avg_white_alc)

'Success'

**Question 1.7** <br> {points: 1}  

Plot a bar chart showing the quantity of each wine style in the `train_df` dataframe. Make sure to give it a title and name your plot style_prop. 

In [16]:
style_prop = alt.Chart(train_df).mark_bar().encode(x=alt.X('style:N', title='Style'), y=alt.Y('count():Q', title='Quantity')).properties(width=300, height=350, title = 'Wine Quantity by Style')
style_prop

alt.Chart(...)

In [17]:
t.test_1_7(style_prop)

'Success'

## 2. Finding the Nearest Neighbours

Let's explore the training set a bit more and calculate the distance between examples.

**Question 2.1** <br> {points: 1}  

Split up `train_df` and `test_df` into feature and target object and name them respectively `X_train`, `y_train`, `X_test` and `y_test`. Remember that our target value is `style`.

In [18]:
train_df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,style
2415,8.4,0.18,0.42,5.10,0.036,7.0,77.0,0.99390,3.16,0.52,11.7,5,white
4861,7.6,0.19,0.32,18.75,0.047,32.0,193.0,1.00014,3.10,0.50,9.3,7,white
2413,7.9,0.35,0.24,15.60,0.072,44.0,229.0,0.99785,3.03,0.59,10.5,6,white
3366,7.4,0.32,0.22,1.70,0.051,50.0,179.0,0.99550,3.28,0.69,8.9,5,white
2944,7.3,0.39,0.37,1.10,0.043,36.0,113.0,0.99100,3.39,0.48,12.7,8,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,7.0,0.47,0.07,1.10,0.035,17.0,151.0,0.99100,3.02,0.34,10.5,5,white
2139,6.7,0.31,0.31,9.90,0.040,10.0,175.0,0.99530,3.46,0.55,11.4,4,white
3779,8.0,0.36,0.43,10.10,0.053,29.0,146.0,0.99820,3.40,0.46,9.5,6,white
4488,8.4,0.36,0.36,11.10,0.032,21.0,132.0,0.99313,2.95,0.39,13.0,6,white


In [19]:
X_train = train_df.iloc[:,0:12]
y_train = train_df.iloc[:,12]
X_test = test_df.iloc[:,0:12]
y_test = test_df.iloc[:,12]

In [20]:
t.test_2_1(X_train,X_test,y_train,y_test)

'Success'

**Question 2.2** <br> {points: 1}  

What are the distances between all the wines in the training set? Save this in an object named `wine_similarities`.

*Hint: Make sure you are importing the necessary library.*

In [21]:
from sklearn.metrics.pairwise import euclidean_distances
wine_similarities = euclidean_distances(X_train)
wine_similarities

array([[  0.        , 119.48948389, 156.79931254, ...,  72.6367263 ,
         57.09454454,  11.85275469],
       [119.48948389,   0.        ,  38.11165347, ...,  47.89733959,
         62.5772952 , 110.52251177],
       [156.79931254,  38.11165347,   0.        , ...,  84.53076577,
         99.82390757, 147.46605787],
       ...,
       [ 72.6367263 ,  47.89733959,  84.53076577, ...,   0.        ,
         16.54154668,  63.03668437],
       [ 57.09454454,  62.5772952 ,  99.82390757, ...,  16.54154668,
          0.        ,  48.27779763],
       [ 11.85275469, 110.52251177, 147.46605787, ...,  63.03668437,
         48.27779763,   0.        ]])

In [22]:
t.test_2_2(wine_similarities)

'Success'

**Question 2.3** <br> {points: 1}  

Which wine index is most similar to that at index 12? Save the index in an object named `sim_wine12`.

*Hint: You'll need to make sure you use `fill_diagonal()`.*

In [23]:
pd.DataFrame(wine_similarities)
np.fill_diagonal(wine_similarities, np.inf)
sim_wine12 = np.argmin(wine_similarities[12])

In [24]:
t.test_2_3(sim_wine12)

'Success'

**Question 2.4** <br> {points: 2}  

What is the distance between the wine at `sim_wine12` and index 12? 

Save this in an object named `distance_12`.

In [25]:
distance_12 = wine_similarities[12][sim_wine12]
distance_12

2.2361352821281204

In [26]:
# check that the variable exists
assert 'distance_12' in globals(
), "Please make sure that your solution is named 'distance_12'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.

**Question 2.5** <br> {points: 1}  

A new wine was just released into liquor stores with the following feature vector.

```
[ 8.3   ,  0.325 ,  0.36  ,  13.3    ,  0.101 , 23.    , 49.    ,
        0.9966,  3.56  ,  0.42  , 8.2    ,  7.    ]
```

Which wine from the training dataset should wine merchants mention that it is most similar to? 


Save this in an object named `similar_new_wine`.

In [27]:
from sklearn.neighbors import NearestNeighbors
new_wine = [[8.3, 0.325, 0.36, 13.3, 0.101, 23.0, 49.0, 0.9966, 3.56, 0.42, 8.2, 7.0]]
nn = NearestNeighbors(n_neighbors=5)
nn.fit(X_train)
distance = euclidean_distances(X_train, new_wine)
similar_new_wine = np.argmin(distance)

In [28]:
t.test_2_5(similar_new_wine)

'Success'

**Question 2.6** <br> {points: 1}  

How far away is the new wine from the most similar wine in the training dataset? 

Save this distance in an object named `new_distance`.

In [29]:
new_distance = distance[np.argmin(distance)].item()
new_distance

7.225013265704105

In [30]:
t.test_2_6(new_distance)

'Success'

## 3. KNN Classifiers with different hyperparameters 

**Question 3.1** <br> {points: 1}  

Build a `DummyClassifier` using `strategy = 'most_frequent'` and name it `dummy_model`.

Train it on `X_train` and `y_train`. Score it on the train **and** test sets.

Save the scores in objects named `dummy_train` and `dummy_test`.


In [31]:
dummy_model = DummyClassifier(strategy = 'most_frequent')

dummy_model.fit(X_train, y_train).predict(X_train)
dummy_train = dummy_model.score(X_train, y_train)

dummy_model.fit(X_test, y_test).predict(X_test)
dummy_test = dummy_model.score(X_test, y_test)

print(dummy_train)
print(dummy_test)

0.7527419665191457
0.7584615384615384


In [32]:
t.test_3_1(dummy_train,dummy_test,dummy_model)

'Success'

**Question 3.2** <br> {points: 1} 

Build a `KNeighborsClassifier` named `knn1` with  `n_neighbors=1`. Cross-validate using `cv=10`. 
What is the mean training score and the mean validation score? Save each respectively in objects named `knn1_train_score` and `knn1_valid_score`.

In [33]:
from sklearn.model_selection import cross_val_score

knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train, y_train).predict(X_train)
knn1_train_score = knn1.score(X_train, y_train).mean()
knn1_valid_score = knn1.score(X_test, y_test).mean()

print(knn1_train_score)
print(knn1_valid_score)

0.9996151625938041
0.95


In [34]:
t.test_3_2(knn1_train_score,knn1_valid_score,knn1)

'Success'

**Question 3.3** <br> {points: 2} 

Which model has the best training accuracy? 

A) `DummyClassifier`. 

B) `KNeighborsClassifier(n_neighbors=1)`. 

C) Both A and B

*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer3_3`.*


In [35]:
answer3_3 = 'B'
answer3_3

'B'

In [36]:
# check that the variable exists
assert 'answer3_3' in globals(
), "Please make sure that your solution is named 'answer3_3'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.


**Question 3.4** <br> {points: 1} 

Which model has the best cross-validation accuracy?

A) `DummyClassifier`. 

B) `KNeighborsClassifier(n_neighbors=1)`. 

C) Both A and B

*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer3_4`.*


In [37]:
answer3_4 = 'B'
answer3_4

'B'

In [38]:
t.test_3_4(answer3_4)

'Success'

**Question 3.5** <br> {points: 1}

Which model is probably overfitting?

A) `DummyClassifier`

B) `KNeighborsClassifier(n_neighbors=1)`

C) Both A and B

*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer3_5`.*


In [39]:
answer3_5 = 'B'
answer3_5

'B'

In [40]:
t.test_3_5(answer3_5)

'Success'

**Question 3.6** <br> {points: 1} 

***True or False*** 

For smaller values of $k$ you are expected to get higher training scores. 


*Answer in the cell below by assigning `True` or `False` to an object called `answer3_6`.*

In [41]:
answer3_6 = True
answer3_6

True

In [42]:
t.test_3_6(answer3_6)

'Success'

**Question 3.7** <br> {points: 1} 

If we increase the number of features, which of the following is true?

A) The training and validation scores will always increase.

B) The training and validation scores will always decrease.

C) The training and validation scores will decrease when we start adding irrelevant features.

D) The model only picks features that it deems important so nothing changes. 

*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer3_7`.*

In [43]:
answer3_7 = 'C'
answer3_7

'C'

In [44]:
t.test_3_7(answer3_7)

'Success'

**Question 3.8** <br> {points: 3} 

Now let's do some hyperparameter tuning and find the most optimal value for  `n_neighbors` in our `KNeighborsClassifier`. 

We want to find the best hyperparameter value to predict on our test set so let's build a loop as we have in the previous assignments where we record the training and cross-validation scores for each hyperparameter value.

Create a `for` loop that iterates over `n_neighbors` values from every second number from 2 to 20 (inclusive). We've started this for you.

Each iteration should:
1. Create a `KKNeighborsClassifier` object with `n_neighbors` changing at each iteration.
2. Run 5-fold cross-validation with this value of `n_neighbors` using `cross_validate` to get the mean train and validation accuracies. Make sure to set `return_train_score=True` to get the training score in each fold. 
3. Appends the `n_neighbors` value to the list in the key `n_neighbors` of the dictionary named `results_dict`.
4. Appends the mean `train_score` of the cross-validation folds to the list in the `mean_train_score` dictionary key. 
5. Appends the mean `test_score` of the cross-validation folds to the list in the `mean_cv_score` dictionary key. 

(Note that this may take a few minutes to execute)

In [45]:
results_dict = {
    "n_neighbors": [],
    "mean_train_score": [],
    "mean_cv_score": []}

results_dict

for k in range(2,20, 2):
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(model, X_train, y_train, cv=5, return_train_score=True)
    results_dict["n_neighbors"].append(k)
    results_dict["mean_cv_score"].append(scores["test_score"].mean())
    results_dict["mean_train_score"].append(scores["train_score"].mean())    
    
results_dict

{'n_neighbors': [2, 4, 6, 8, 10, 12, 14, 16, 18],
 'mean_train_score': [0.9723396721953372,
  0.9588224131644868,
  0.9536269368600377,
  0.9484317498269867,
  0.9457379272871215,
  0.9442467563708844,
  0.9423707156447112,
  0.9418897035928548,
  0.9409756869703946],
 'mean_cv_score': [0.9274576145702229,
  0.9326530687791514,
  0.9397721551787962,
  0.9399648330495299,
  0.9405421263048789,
  0.9391952321018732,
  0.9390027393203525,
  0.9384254460650034,
  0.9372715999111572]}

In [46]:
t.test_3_8(results_dict)

'Success'

**Question 3.9** <br> {points: 1} 

Convert the dictionary `results_dict` into a dataframe and use `pd.melt()` to melt the columns `mean_train_score` and `mean_cv_score` in the `results_df`.  Use `var_name='score_type'` and `value_name='accuracy'` and name the new dataframe `knn_plot_df`. 

In [47]:
results_df = pd.DataFrame(results_dict)
knn_plot_df = results_df.melt(id_vars=['n_neighbors'], value_vars=['mean_cv_score','mean_train_score'], var_name='score_type', value_name='accuracy')

In [48]:
t.test_3_9(knn_plot_df)

'Success'

**Question 3.10** <br> {points: 1} 

Using Altair, make a `mark_line()` plot which displays the `n_neighbors` of the KNN model on the *x*-axis and the accuracy on the train and validation sets on the *y*-axis and don't forget to add `alt.Color(score_type)` to the `encode()` function after you specify `alt.X()` and `alt.y()`. 

Make sure it has the dimensions `width=500, height=300`. Don't forget to give it a title and the plot `knn_plot`.
To make things more legible, use `scale=alt.Scale(domain=[.92, 0.98])` in your `alt.Y()` function which will start the y-axis at 0.92 and end it at 0.98. 

In [49]:
knn_plot = alt.Chart(knn_plot_df).mark_line().properties(width=500, height=300, title='knn_plot').encode(x=alt.X('n_neighbors:Q'), y=alt.Y('accuracy:Q', scale=alt.Scale(domain=[.92, 0.98])), color=alt.Color('score_type'))
knn_plot

alt.Chart(...)

In [50]:
t.test_3_10(knn_plot)

'Success'

**Question 3.11** <br> {points: 1} 

From your results, what `n_neighbors` would you pick in your final model? Save your answer in an object named `best_k`.

*Hint: [<code>.idxmax()</code>](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.idxmax.html) may come in handy.*

In [51]:
best_k = results_df.loc[results_df['mean_cv_score'].idxmax()]['n_neighbors']
best_k

10.0

In [52]:
t.test_3_11(best_k)

'Success'

**Question 3.12** <br> {points: 1} 

Build a K-Nearest Neighbour classifier named `best_model` with the best `n_neighbors` and fit it with `X_train` and `y_train`. Score your model on the test set and save your results in an object named `test_score`.

Is this doing better than your dummy classifier?

In [83]:
best_model = KNeighborsClassifier(int(best_k)).fit(X_train,y_train)
test_score = best_model.score(X_test,y_test)
test_score

0.9353846153846154

In [54]:
t.test_3_12(test_score)

'Success'

# 4. Support Vector Machines Classifier 

Up until this point, we have been working with the K-Nearest Neighbour Classifier. Let's shake things up a bit and explore the second model that we learned in this module. Unlike other questions up to this point, we've only explored one hyperparameter at a time. This time let's see how well we can optimize more than one hyperparameter simultaneously.

**Question 4.1** <br> {points: 0} 

Import SVC from the appropriate library. 

In [55]:
from sklearn.svm import SVC

In [56]:
t.test_4_1()

'Success'

**Question 4.2** <br> {points: 1} 

Repeat the loop you made in **Question 3.8** but this time but this time, tune the hyperparameter `gamma` and find the most optimal value for  `gamma` in an `SVC` model.  

We want to find the best gamma value to test our model.
Don't forget to set `random_state=2020` so we can confirm your answer. 

Create a `for` loop that iterates over `gamma` values from 0.1 to 100 (inclusive) that increases exponentially with base 10. (We've started this for you.)

To recap the instructions from before, each iteration should:
1. Create a `SVC` object with `gamma` changing at each iteration.
2. Run 5-fold cross-validation with this value of `gamma` using `cross_validate` to get the mean train and validation accuracies. Make sure to set `return_train_score=True` to get the training score in each fold. 
3. Appends the `gamma` value to the list in the key `gamma`.
4. Appends the mean `train_score` of the cross-validation folds to the list in the `mean_train_score` dictionary key. 
5. Appends the mean `test_score` of the cross-validation folds to the list in the `mean_cv_score` dictionary key. 

(Note that this may take quite a few minutes to execute)

In [57]:
results_dict = {
    "gamma": [],
    "mean_train_score": [],
    "mean_cv_score": []}

results_dict

for g in [0.1, 1.0, 10.0, 100.0]:
    model = SVC(gamma=g, random_state = 2020)
    scores = cross_validate(model, X_train, y_train, cv=5, return_train_score=True)
    results_dict["gamma"].append(g)
    results_dict["mean_cv_score"].append(scores["test_score"].mean())
    results_dict["mean_train_score"].append(scores["train_score"].mean())    

results_dict

{'gamma': [0.1, 1.0, 10.0, 100.0],
 'mean_train_score': [0.9790263425577355,
  0.9996632533798759,
  0.9996632533798759,
  0.9996632533798759],
 'mean_cv_score': [0.9447745613385653,
  0.8170098837639742,
  0.8027702302509809,
  0.8016156437402827]}

In [58]:
t.test_4_2(results_dict)

'Success'

**Question 4.3** <br> {points: 1} 

Which value of gamma would you select for your model. Save your result in an object named `best_gamma`.


In [59]:
results_df = pd.DataFrame(results_dict)
best_gamma = results_df.loc[results_df['mean_cv_score'].idxmax()]['gamma']
best_gamma

0.1

In [60]:
t.test_4_3(best_gamma)

'Success'

**Question 4.4** <br> {points: 1} 

Now repeat **Question 4.2**, this time iterating over the hyperparameter C. 

In [61]:
results_dict = {
    "C": [],
    "mean_train_score": [],
    "mean_cv_score": []}

results_dict

for c in [0.1, 1.0, 10.0, 100.0]:
    model=SVC(C=c, random_state=2020)
    scores = cross_validate(model, X_train, y_train, cv=5, return_train_score=True)
    results_dict["C"].append(c)
    results_dict["mean_cv_score"].append(scores["test_score"].mean())
    results_dict["mean_train_score"].append(scores["train_score"].mean())    

results_dict

{'C': [0.1, 1.0, 10.0, 100.0],
 'mean_train_score': [0.9305849657786152,
  0.9369829201438534,
  0.9478544798246507,
  0.9578121385915468],
 'mean_cv_score': [0.9303451913822463,
  0.9368873547049678,
  0.9474698304582809,
  0.9572827052639372]}

In [62]:
t.test_4_4(results_dict)

'Success'

**Question 4.5** <br> {points: 1} 

Which value of `C` would you select for your model now? Save your result in an object named `best_c`.

In [63]:
results_df = pd.DataFrame(results_dict)
best_c = results_df.loc[results_df['mean_cv_score'].idxmax()]['C']
best_c

100.0

In [64]:
t.test_4_5(best_c)

'Success'

**Question 4.6** <br> {points: 2} 

Do you think choosing the value of `Gamma` from question 4.3 and the value of `C` from question 4.5 will produce the best scoring model to run our test set on?  

A) No. we should have set `gamma` to `best_gamma` and iterated over the value of `C`. This would have produced the most optimal model. 

B) No, we should be searching all values of gamma with all values of C since it's possible that hyperparameter values may score lower independently but could score higher when cross-validated together. 

C) Yes. Both `Gamma` and `C` produced the highest cross-validation scores independently and therefore they must produce the highest cross-validation scores together.   

D) Yes. `Gamma` and `C` are not correlated and therefore finding the best values separately will not change how the scores would be if we tested them together. 


*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer4_6`.*


In [65]:
answer4_6 = 'B'
answer4_6

'B'

In [66]:
# check that the variable exists
assert 'answer4_6' in globals(
), "Please make sure that your solution is named 'answer4_6'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.

**Question 4.7** <br> {points: 2} 

Write a nested loops to search over gamma and C simultaneously. Use 5 fold cross-validation and append the training and validation (`test_score`) scores to the `param_scores` dictionary. 

*Note: This could take quite a few minutes to run.*

In [67]:
hyperparameters = {
    "gamma": [0.1, 1.0, 10.0, 100.0],
    "C": [0.1, 1.0, 10.0, 100.0]
}
param_scores = {"gamma": [], "C": [], "train_accuracy": [], "valid_accuracy": []}

for g in [0.1, 1.0, 10.0, 100.0]:
    for c in [0.1, 1.0, 10.0, 100.0]:
        model=SVC(gamma=g, C=c, random_state=2020)
        scores = cross_validate(model, X_train, y_train, cv=5, return_train_score=True)
        param_scores["gamma"].append(g)
        param_scores["C"].append(c)
        param_scores["valid_accuracy"].append(scores["test_score"].mean())
        param_scores["train_accuracy"].append(scores["train_score"].mean())    

In [68]:
t.test_4_7(param_scores)

'Success'

**Question 4.8** <br> {points: 1} 

Save `param_scores` as a dataframe named `param_scores_df` and sort by validation score in descending order. 


In [76]:
param_scores_df = pd.DataFrame(param_scores).sort_values(by='valid_accuracy', ascending=False)
param_scores_df

,gamma,C,train_accuracy,valid_accuracy
3,0.1,100.0,0.999663,0.951126
2,0.1,10.0,0.999663,0.950933
1,0.1,1.0,0.979026,0.944775
0,0.1,0.1,0.871176,0.863768
6,1.0,10.0,0.999663,0.825092
7,1.0,100.0,0.999663,0.825092
5,1.0,1.0,0.999663,0.817010
10,10.0,10.0,0.999663,0.803348
11,10.0,100.0,0.999663,0.803348
9,10.0,1.0,0.999663,0.802770


In [74]:
t.test_4_8(param_scores_df)

'Success'

**Question 4.9** <br> {points: 1} 

Build your new model name `best_svc` using the values for `gamma` and `C` we obtained when tunning then hyperparameters simultaneously in **Question 4.8**. Set the random_state to 2020 and fit it with `X_train` and `y_train`. Score your model on the test set and save your results in an object named `svc_test_score`. 

In [81]:
svc_test_score = None 

best_svc = SVC(gamma=0.1, C=100, random_state=2020).fit(X_train, y_train)
svc_test_score = best_svc.score(X_test,y_test)

In [82]:
t.test_4_9(svc_test_score)

'Success'

**Question 4.10** <br> {points: 1} 

Which model performs better?

A) `KNeighborsClassifier`

B) `SVC`

*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer4_10`.*


In [84]:
answer4_10 = 'B'

In [85]:
t.test_4_10(answer4_10)

'Success'

## Before Submitting 

Before submitting your assignment please do the following:

- Read through your solutions
- **Restart your kernel and clear output and rerun your cells from top to bottom** 
- Makes sure that none of your code is broken 
- Verify that the tests from the questions you answered have obtained the output "Success"

This is a simple way to make sure that you are submitting all the variables needed to mark the assignment. This method should help avoid losing marks due to changes in your environment.  

## Attributions
- Wine dataset - [Kaggle](https://www.kaggle.com/numberswithkartik/red-white-wine-dataset)


- MDS DSCI 571 - Supervised Learning I - [MDS's GitHub website](https://github.com/UBC-MDS/DSCI_571_sup-learn-1) 
